## Modele de clasificare
Folositi 4 seturi de date pentru probleme de clasificare, plecand de la repository-urile specificate in Cursul 6. Cel putin un set de date sa fie cu valori lipsa; pentru un alt set de date care are initial toate valorile, introduceti dvs. in mod artificial valori lipsa, suprascriind un anumit procent din valorile initiale (ex. p=5%, p parametru) cu numpy.nan.

In [463]:
import numpy as np
print(f'NumPy version: {np.__version__}')
import pandas as pd
print(f'Pandas version: {pd.__version__}')
import sklearn as sk
print(f'Sklearn version: {sk.__version__}')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
#NumPy version: 1.20.1
#Pandas version: 1.2.3
#Sklearn version: 0.24.1

NumPy version: 1.20.1
Pandas version: 1.2.3
Sklearn version: 0.24.1


In [464]:
le=LabelEncoder()

In [465]:
iris: pd.core.frame.DataFrame=pd.read_csv("Data/iris.data",header=None)
x_iris:np.array = iris.iloc[:, :-1].values
y_iris:np.array = iris[[4]].apply(le.fit_transform).values
y_iris=y_iris.reshape(y_iris.shape[0])

In [466]:
wine:pd.core.frame.DataFrame=pd.read_csv("Data/wine.data",header=None)
x_wine:np.array = wine.iloc[:,1:].values
y_wine:np.array = wine.iloc[:,0].values

In [467]:
wdbc:pd.core.frame.DataFrame=pd.read_csv("Data/wdbc.data",header=None)  
x_wdbc:np.array = wdbc.iloc[:, 2:].values
y_wdbc:np.array = wdbc[[1]].apply(le.fit_transform).values
y_wdbc=y_wdbc.reshape(y_wdbc.shape[0])

In [468]:
abalone:pd.core.frame.DataFrame=pd.read_csv("Data/abalone.data",header=None) 
x_abalone:np.array = abalone.iloc[:, 1:].values
y_abalone:np.array = abalone[[0]].apply(le.fit_transform).values
y_abalone=y_abalone.reshape(y_abalone.shape[0])

1. (20 puncte) Aplicati o metoda de missing value imputation, unde este cazul; documentati metoda folosita.
*Resurse*: Pentru missing value imputation, puteti urmari [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html), [How to Handle Missing Data with Python](https://machinelearningmastery.com/handle-missing-data-python/), [fancyimpute](https://github.com/iskandr/fancyimpute), [missingpy](https://github.com/epsilon-machine/missingpy).

*Cerinta*: In cazul in care folositi un pachet ce trebuie instalat (nu face parte din distributia standard anaconda), includeti intr-o celula o comanda de instalare corespunzatoare folosind semn de exclamare, de exemplu:
```python
!pip install missingpy
```
(sursa: [https://github.com/epsilon-machine/missingpy](https://github.com/epsilon-machine/missingpy)). La executia celulei in Jupyter Notebook se instaleaza pachetul, iar in celulele ulterioare importurile din noul pachet functioneaza. 

In [469]:
def Random_Nans(data_set:np.array,percentage:int = 5)->np.array:
    """
    Adding NaN values in a dataset.
    Args:
        data_set:the numpy.ndarray containing the dataset
        percentage:the percentage(from dataset size) of Nans values to be added in the dataset
    Returns:the modified dataset
    """
    result_set:np.array=data_set.copy()
    choice:np.array = np.random.choice(result_set.size, int(percentage / 100 * result_set.size), replace=False)
    rows,cols= np.unravel_index(choice, result_set.shape)   
    result_set[rows, cols] = np.nan
    return result_set

In [470]:
x_iris=Random_Nans(x_iris)
x_wine=Random_Nans(x_wine)
x_wdbc=Random_Nans(x_wdbc)
x_abalone=Random_Nans(x_abalone)

In [471]:
imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
x_iris=imp.fit_transform(x_iris)
x_wine=imp.fit_transform(x_wine)
x_wdbc=imp.fit_transform(x_wdbc)
x_abalone=imp.fit_transform(x_abalone)

2. (numar de modele * numar de seturi de date \* 1 punct = 20 de puncte) Pentru fiecare set de date aplicati 5 modele de clasificare din scikit learn. Pentru fiecare raportati: acuratete, precision, recall, scorul F1 - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics), [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) - folosind 5 fold cross validation. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare. Rularile se vor face cu valori fixate ale hiperparametrilor. 

In [472]:
x_iris_train, x_iris_test, y_iris_train, y_iris_test =train_test_split(
    x_iris, y_iris, test_size=1/3, random_state=20,shuffle=True)

x_wine_train, x_wine_test, y_wine_train, y_wine_test = train_test_split(
    x_wine, y_wine, test_size=1/3, random_state=20,shuffle=True)

x_wdbc_train, x_wdbc_test, y_wdbc_train, y_wdbc_test = train_test_split(
    x_wdbc, y_wdbc, test_size=1/3, random_state=20,shuffle=True)

x_abalone_train, x_abalone_test, y_abalone_train, y_abalone_test = train_test_split(
    x_abalone, y_abalone, test_size=1/3, random_state=20,shuffle=True)

In [473]:
def KNeighbours(x,y,n_neighbors=5):
    model_5 = KNeighborsClassifier(n_neighbors=n_neighbors)
    model_5.fit(x, y)
    y_pred = model_5.predict(x)
    return y_pred

In [474]:
print(f'KNeighbours:{round(accuracy_score(y_iris_train,KNeighbours(x_iris_train,y_iris_train)) * 100, 2)}% accuracy for iris_train')
print(f'KNeighbours:{round(accuracy_score(y_wine_train,KNeighbours(x_wine_train,y_wine_train)) * 100,2)}% accuracy for wine_train')
print(f'KNeighbours:{round(accuracy_score(y_wdbc_train,KNeighbours(x_wdbc_train,y_wdbc_train)) * 100,2)}% accuracy for wdbc_train')
print(f'KNeighbours:{round(accuracy_score(y_abalone_train,KNeighbours(x_abalone_train,y_abalone_train)) * 100,2)}% accuracy for abalone_train')

KNeighbours:96.0% accuracy for iris_train
KNeighbours:82.2% accuracy for wine_train
KNeighbours:94.72% accuracy for wdbc_train
KNeighbours:67.21% accuracy for abalone_train


In [475]:
print(f'KNeighbours:{round(accuracy_score(y_iris_test,KNeighbours(x_iris_test,y_iris_test)) * 100, 2)}% accuracy for iris_test')
print(f'KNeighbours:{round(accuracy_score(y_wine_test,KNeighbours(x_wine_test,y_wine_test)) * 100,2)}% accuracy for wine_test')
print(f'KNeighbours:{round(accuracy_score(y_wdbc_test,KNeighbours(x_wdbc_test,y_wdbc_test)) * 100,2)}% accuracy for wdbc_test')
print(f'KNeighbours:{round(accuracy_score(y_abalone_test,KNeighbours(x_abalone_test,y_abalone_test)) * 100,2)}% accuracy for abalone_test')

KNeighbours:92.0% accuracy for iris_test
KNeighbours:70.0% accuracy for wine_test
KNeighbours:93.16% accuracy for wdbc_test
KNeighbours:64.97% accuracy for abalone_test


In [495]:
print(f'KNeighbours:{precision_score(y_iris_train,KNeighbours(x_iris_train,y_iris_train,5),average=None) * 100} precision for iris_train')
print(f'KNeighbours:{precision_score(y_wine_train,KNeighbours(x_wine_train,y_wine_train,5),average=None) * 100} precision for wine_train')
print(f'KNeighbours:{precision_score(y_wdbc_train,KNeighbours(x_wdbc_train,y_wdbc_train,5),average=None) * 100} precision for wdbc_train')
print(f'KNeighbours:{precision_score(y_abalone_train,KNeighbours(x_abalone_train,y_abalone_train,5),average=None) * 100} precision for abalone_train')

KNeighbours:[ 91.66666667 100.          97.05882353] precision for iris_train
KNeighbours:[85.10638298 78.26086957 84.        ] precision for wine_train
KNeighbours:[94.69387755 94.7761194 ] precision for wdbc_train
KNeighbours:[57.67590618 76.65975104 67.00680272] precision for abalone_train


In [487]:
def LRegression(x,y):
    logreg = LogisticRegression(solver='sag', multi_class='multinomial', max_iter=10000)
    logreg.fit(x, y)
    y_pred = logreg.predict(X_test)
    return y_pred